<a href="https://colab.research.google.com/github/s3xmaxman/whisper-translate/blob/main/whisper_srt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U openai-whisper
!pip install srt

In [ ]:
import whisper
from datetime import timedelta
import srt
from srt import Subtitle
import subprocess


# 音声データ抽出 (音声ファイル名変更)
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

# Whisperモデル読み込み
model = whisper.load_model("large-v3")

# 音声ファイル読み込み
audio = whisper.load_audio("audio.mp3")

# 音声認識 (開始・終了時間付きセグメント取得)
result = model.transcribe(audio, verbose=True)
segments = result["segments"]

# SRTファイル用の空リスト初期化
subtitles = []

# セグメントごとにSRT行を作成
for index, segment in enumerate(segments, start=1):
    start_time = timedelta(seconds=segment["start"])
    end_time = timedelta(seconds=segment["end"])
    text = segment["text"]

    subtitle = Subtitle(
        index=index,
        start=start_time,
        end=end_time,
        content=text
    )
    subtitles.append(subtitle)

# SRTファイル内容の作成
srt_content = srt.compose(subtitles)

# ファイルへの書き込み (エンコーディング指定)
with open("output.srt", "w", encoding="utf-8") as f:
    f.write(srt_content)